<a href="https://colab.research.google.com/github/Neetagrg/Mini-Project_2_Spring-2025-WEB-DATA-MINING-CUS-635/blob/main/Mini_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mini Project **2**: Neeta Kumari & Bir Bahadur Gharti Class CUS635

# Install Required Libraries

In [ ]:

!pip install boto3 requests pandas

# Import Required Libraries

In [22]:
# Import the boto3 library
import boto3
from botocore.config import Config
from botocore import UNSIGNED

# Team 6 folder
TEAM = "TEAM_6/"

# Define the provided S3 bucket name
BUCKET_NAME = "cus635-spring2025"

# Create an anonymous S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Upload the CSV to S3 Download a File from S3

In [26]:
import boto3
from botocore.config import Config
from botocore import UNSIGNED

# Team
TEAM = "TEAM_6/"
BUCKET_NAME = "cus635-spring2025"

# Create an anonymous S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Download a file from S3
object_name = "TEAM_6/Financial_Articles.csv"  # Corrected file path
new_name = "downloaded_Finance_Articles.csv"

s3.download_file(BUCKET_NAME, object_name, new_name)
print(f"File '{object_name}' downloaded successfully as '{new_name}'!")


File 'TEAM_6/Financial_Articles.csv' downloaded successfully as 'downloaded_Finance_Articles.csv'!


## Fetch & Save 50 Finance Articles

In [34]:
import openai
import pinecone
import requests
import pandas as pd
from tqdm import tqdm

# Guardian API configuration
API_URL = "https://content.guardianapis.com/search"
API_KEY = "80d8126f-3f17-4f76-ab02-e27428b23e63"  # Replace with your actual API key

# Settings
PAGE_SIZE = 25   # 25 results per page
MAX_ARTICLES = 50  # Target is 50 articles
total_articles = []

# Function to fetch articles
def fetch_articles(page_num):
    params = {
        "api-key": API_KEY,
        "section": "business",  # Finance-related section
        "q": "",  # Removing the specific query for AI in Finance
        "page-size": PAGE_SIZE,
        "page": page_num,
        "show-fields": "body"
    }
    try:
        response = requests.get(API_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {page_num}: {e}")
        return None

# Function to process each article
def process_articles(articles_data):
    if "response" not in articles_data or "results" not in articles_data["response"]:
        print("Invalid response format")
        return

    for article in articles_data["response"]["results"]:
        heading = article.get("webTitle", "No title available")
        context = article.get("fields", {}).get("body", "No content available")
        cleaned_context = (context[:500] + "...") if len(context) > 500 else context.strip()
        total_articles.append({"heading": heading, "context": cleaned_context})

# Fetch enough pages to reach 50 articles
page = 1
with tqdm(desc="Fetching pages", unit="page") as pbar:
    while len(total_articles) < MAX_ARTICLES:
        data = fetch_articles(page)
        if data:
            process_articles(data)
            pbar.update(1)
        else:
            print(f"Error on page {page}, skipping...")
        page += 1

# Only keep the first 50 articles
df = pd.DataFrame(total_articles[:MAX_ARTICLES])

# Save to CSV
csv_filename = "Finance_Articles.csv"  # You can change the filename if needed
df.to_csv(csv_filename, index=False)

# Preview & confirmation
print(df.head())
print(f"✅ Fetched and saved {len(df)} articles to '{csv_filename}'")

# Now proceed with upsert to Pinecone

# Initialize Pinecone
pinecone.init(api_key="your-pinecone-api-key", environment="your-environment")

# Create the index if it doesn't exist
index_name = "article-index"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536, metric="cosine")

# Connect to the index
index = pinecone.Index(index_name)

# Prepare data for upsert
vectors_to_upsert = []

# Use OpenAI to embed the article contents
openai.api_key = "your-openai-api-key"

for i, article in enumerate(total_articles):
    heading = article["heading"]
    context = article["context"]

    # Combine heading and context for embedding
    article_text = heading + " " + context

    # Use OpenAI API to get embedding (this will depend on the embedding model you are using)
    response = openai.Embedding.create(input=article_text, model="text-embedding-ada-002")
    embedding = response["data"][0]["embedding"]

    # Prepare data for upsert
    vectors_to_upsert.append({
        "id": f"article-{i+1}",
        "values": embedding,
        "metadata": {"text": article_text}
    })

# Perform the upsert to Pinecone
index.upsert(vectors=vectors_to_upsert)

print("✅ Articles embedded and upserted successfully.")


Fetching pages: 2page [00:02,  1.10s/page]

                                             heading  \
0  Ignore Turnbull's rhetoric on income tax cuts ...   
1  Gina Rinehart awards Barnaby Joyce $40,000 for...   
2  Furniture maker Multiyork enters administratio...   
3  For the UK economy, this budget is its Suez mo...   
4  UK water firms admit using divining rods to fi...   

                                             context  
0  <p>The rather unexpected allusion by the prime...  
1  <p>Australia’s richest woman, mining magnate G...  
2  <p>Multiyork Furniture has called in administr...  
3  <p>The 1956 <a href="https://www.theguardian.c...  
4  <p>Ten of the 12 water companies in the UK hav...  
✅ Fetched and saved 50 articles to 'Finance_Articles.csv'


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



# Load and Preprocess Articles

In [29]:
import pandas as pd

# Load the CSV containing the articles
df = pd.read_csv("downloaded_Finance_Articles.csv")

# Preview the data to ensure it's loaded correctly
print(df.head())


                                             heading  \
0  Motor finance compensation ruling ‘goes too fa...   
1  Supreme court judges reject Reeves’ motor fina...   
2  Car finance firm Close Brothers slumps to loss...   
3  Chief finance officer of under-fire Thames Wat...   
4  Car finance scandal: lender sets aside £165m f...   

                                             context  
0  <p>A court of appeal ruling that has left lend...  
1  <p>Rachel Reeves was dealt a fresh blow on Mon...  
2  <p>Close Brothers, one of the UK’s biggest pro...  
3  <p>Thames Water’s chief financial officer has ...  
4  <p>One of the UK’s biggest providers of motor ...  


# Pinecone Setup

In [30]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone with your API key and environment
pc = Pinecone(api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6", environment="us-east-1")

# Access your index using the Index method
index = pc.Index("cus635")


In [ ]:
import os
import pinecone
import requests
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from sentence_transformers import SentenceTransformer

# Guardian API configuration
API_URL = "https://content.guardianapis.com/search"
API_KEY = "80d8126f-3f17-4f76-ab02-e27428b23e63"

# Settings
PAGE_SIZE = 50   # Set to maximum allowed page size
MAX_ARTICLES = 50  # Target is 50 articles
TEAM_NAME = "Team 6"
CATEGORY = "Finance"
total_articles = []

# Initialize the embedding model (with correct dimension model)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')  # This model generates 384-dimensional embeddings

# Function to fetch articles
def fetch_articles(page_num):
    params = {
        "api-key": API_KEY,
        "section": CATEGORY.lower(),
        "q": "",
        "page-size": PAGE_SIZE,
        "page": page_num,
        "show-fields": "body,webPublicationDate"
    }
    try:
        response = requests.get(API_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {page_num}: {e}")
        return None

# Function to process each article
def process_articles(articles_data):
    if "response" not in articles_data or "results" not in articles_data["response"]:
        print("Invalid response format")
        return

    for article in articles_data["response"]["results"]:
        heading = article.get("webTitle", "No title available")
        context = article.get("fields", {}).get("body", "No content available")
        publication_date = article.get("webPublicationDate", "No date available")
        cleaned_context = (context[:500] + "...") if len(context) > 500 else context.strip()
        total_articles.append({
            "heading": heading,
            "context": cleaned_context,
            "publication_date": publication_date
        })

# Fetch enough pages to reach 50 articles
page = 1
with tqdm(desc="Fetching pages", unit="page") as pbar:
    while len(total_articles) < MAX_ARTICLES:
        data = fetch_articles(page)
        if data:
            process_articles(data)
            pbar.update(1)
        else:
            print(f"Error on page {page}, skipping...")
        page += 1

# Only keep the first 50 articles
df = pd.DataFrame(total_articles[:MAX_ARTICLES])

# Save to CSV
csv_filename = "Finance_Articles.csv"
df.to_csv(csv_filename, index=False)

# Preview & confirmation
print(df.head())
print(f"✅ Fetched and saved {len(df)} articles to '{csv_filename}'")

# Initialize Pinecone
pinecone.init(api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6", environment="us-east-1-aws")

# Create the index if it doesn't exist (ensure dimension matches the model's embedding)
index_name = "cus635"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=384,  # Dimension for 'all-MiniLM-L6-v2' model (384)
        metric="cosine",
        pod_type="p1",
        replicas=1,
        metadata_config={"indexed": ["text", "team", "category", "publication_date", "title"]}
    )

# Connect to the index
index = pinecone.Index(index_name)

# Prepare data for upsert
vectors_to_upsert = []

for i, article in enumerate(total_articles):
    heading = article["heading"]
    context = article["context"]
    publication_date = article["publication_date"]

    # Combine heading and context for embedding
    article_text = heading + " " + context

    # Generate embedding using the model
    embedding = model.encode(article_text).tolist()

    # Prepare metadata
    metadata = {
        "text": article_text,
        "team": TEAM_NAME,
        "category": CATEGORY,
        "publication_date": publication_date,
        "title": heading
    }

    # Prepare data for upsert
    vectors_to_upsert.append({
        "id": f"article-{i+1}",
        "values": embedding,
        "metadata": metadata
    })

# Perform the upsert to Pinecone
index.upsert(vectors=vectors_to_upsert)

print("✅ Articles upserted successfully.")

# Verify the documents have been uploaded correctly
def verify_documents(index, vectors_to_upsert):
    ids = [vector["id"] for vector in vectors_to_upsert]
    query_response = index.fetch(ids=ids)
    fetched_ids = list(query_response.vectors.keys())
    missing_ids = set(ids) - set(fetched_ids)
    if missing_ids:
        print(f"Warning: The following documents were not found in the index: {missing_ids}")
    else:
        print("All documents verified successfully.")

verify_documents(index, vectors_to_upsert)


Fetching pages: 0page [00:00, ?page/s]

Error fetching page 1: 429 Client Error: Too Many Requests for url: https://content.guardianapis.com/search?api-key=80d8126f-3f17-4f76-ab02-e27428b23e63&section=finance&q=&page-size=50&page=1&show-fields=body%2CwebPublicationDate
Error on page 1, skipping...
Error fetching page 2: 429 Client Error: Too Many Requests for url: https://content.guardianapis.com/search?api-key=80d8126f-3f17-4f76-ab02-e27428b23e63&section=finance&q=&page-size=50&page=2&show-fields=body%2CwebPublicationDate
Error on page 2, skipping...
Error fetching page 3: 429 Client Error: Too Many Requests for url: https://content.guardianapis.com/search?api-key=80d8126f-3f17-4f76-ab02-e27428b23e63&section=finance&q=&page-size=50&page=3&show-fields=body%2CwebPublicationDate
Error on page 3, skipping...
Error fetching page 4: 429 Client Error: Too Many Requests for url: https://content.guardianapis.com/search?api-key=80d8126f-3f17-4f76-ab02-e27428b23e63&section=finance&q=&page-size=50&page=4&show-fields=body%2CwebPublicati